<a href="https://colab.research.google.com/github/tomdaler/repository/blob/main/ML%20Projects/Wine%20Quality/3_RandomForest_DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Wine Quality, using RandomForestRegressor, RandomForestClassificator RandomizedSearchCV

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('winequality-red.csv')

df['quality']= df['quality']-3
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

scaled_features = df.copy()

col_names = ['fixed acidity','free sulfur dioxide','total sulfur dioxide','alcohol','pH','residual sugar']
features = scaled_features[col_names]
df_new = scale.fit_transform(features)

scaled_features[col_names] = df_new


In [6]:
X = scaled_features.drop(['quality'],axis=1)
y = scaled_features['quality']

X=X.drop(['alcohol', 'sulphates','pH','density','total sulfur dioxide'], axis=1)


In [7]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=2)


In [10]:
n_estimators = [5,20,50,100] # number of trees in the random forest
max_features = ['auto', 'sqrt'] # number of features in consideration at every split
max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
min_samples_split = [2, 6, 10] # minimum sample number to split a node
min_samples_leaf = [1, 3, 4] # minimum sample number that can be stored in a leaf node
bootstrap = [True, False] # method used to sample data points

random_grid = {'n_estimators': n_estimators,

'max_features': max_features,

'max_depth': max_depth,

'min_samples_split': min_samples_split,

'min_samples_leaf': min_samples_leaf,

'bootstrap': bootstrap}

RandomForestRegressor

In [15]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

from sklearn.model_selection import RandomizedSearchCV
rf_random = RandomizedSearchCV(estimator = rf,param_distributions = random_grid,
               n_iter = 100, cv = 5, verbose=2, random_state=35, n_jobs = -1)

rf_random.fit(train_X, train_y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      120],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 3, 4],
                                        'min_samples_split': [2, 6, 10],
                                        'n_estimators': [5, 20, 50, 100]},
                   random_state=35, verbose=2)

In [16]:
print ('Random grid: ', random_grid, '\n')
# print the best parameters
print ('Best Parameters: ', rf_random.best_params_, ' \n')

Random grid:  {'n_estimators': [5, 20, 50, 100], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120], 'min_samples_split': [2, 6, 10], 'min_samples_leaf': [1, 3, 4], 'bootstrap': [True, False]} 

Best Parameters:  {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 120, 'bootstrap': True}  



In [17]:
randmf = RandomForestRegressor(n_estimators = 100, min_samples_split = 2, 
                               min_samples_leaf= 1, max_features = 'sqrt', max_depth= 120, bootstrap=True)


randmf.fit( train_X, train_y)
randmf.score(test_X, test_y)

0.2819384899019527

It didn't get better, 0.28

RandomForestClassifiction

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

from sklearn.model_selection import RandomizedSearchCV
rf_random = RandomizedSearchCV(estimator = rf,param_distributions = random_grid,
               n_iter = 100, cv = 5, verbose=2, random_state=35, n_jobs = -1)

rf_random.fit(train_X, train_y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      120],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 3, 4],
                                        'min_samples_split': [2, 6, 10],
                                        'n_estimators': [5, 20, 50, 100]},
                   random_state=35, verbose=2)

In [19]:
print ('Random grid: ', random_grid, '\n')
# print the best parameters
print ('Best Parameters: ', rf_random.best_params_, ' \n')

Random grid:  {'n_estimators': [5, 20, 50, 100], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120], 'min_samples_split': [2, 6, 10], 'min_samples_leaf': [1, 3, 4], 'bootstrap': [True, False]} 

Best Parameters:  {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 120, 'bootstrap': True}  



In [20]:
randmf = RandomForestRegressor(n_estimators = 100, min_samples_split = 2, 
                               min_samples_leaf= 1, max_features = 'sqrt', max_depth= 120, bootstrap=True)


randmf.fit( train_X, train_y)
randmf.score(test_X, test_y)

0.2765021369311992

Still the best option is DecissionTreeClassifier

In [24]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

bag_model = BaggingClassifier(
    base_estimator=RandomForestClassifier(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)
bag_model.fit(train_X, train_y)
bag_model.oob_score_

0.6219839142091153

PCA

In [25]:
from sklearn.decomposition import PCA
pca =PCA(n_components=11)
pca.fit(X)
pca.explained_variance_ratio_

ValueError: ignored